In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Josep
[nltk_data]     Alet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import time
import pandas as pd
import json

In [16]:
docs_path = 'data/tweet_document_ids_map.csv'
with open(docs_path) as fp:
    lines = fp.readlines()
lines = [l.strip().replace(' +', ' ') for l in lines]
print("Total number of tweet documents in the corpus: {}".format(len(lines)))

Total number of tweet documents in the corpus: 4000


In [125]:
tweets_path = 'data/tw_hurricane_data.json'
with open(tweets_path) as fp:
    tweets = fp.readlines()
tweets = [t for t in tweets]
print("Total number of tweets in the corpus: {}".format(len(tweets)))

Total number of tweets in the corpus: 4001


In [131]:
print(tweets[4000])

In [133]:
tweets_json=[]
i=0
for t in tweets:
    if i != 4000:
        tweets_json.append(json.loads(t)) #The element 4000 is an empty string, if we don't avoid it the code breaks
    i+=1

In [135]:
dt_tweets = pd.DataFrame.from_dict(data=tweets_json)
print(dt_tweets.head())

                       created_at                   id               id_str  \
0  Fri Sep 30 18:39:08 +0000 2022  1575918182698979328  1575918182698979328   
1  Fri Sep 30 18:39:01 +0000 2022  1575918151862304768  1575918151862304768   
2  Fri Sep 30 18:38:58 +0000 2022  1575918140839673873  1575918140839673873   
3  Fri Sep 30 18:38:57 +0000 2022  1575918135009738752  1575918135009738752   
4  Fri Sep 30 18:38:53 +0000 2022  1575918119251419136  1575918119251419136   

                                           full_text  truncated  \
0  So this will keep spinning over us until 7 pm…...      False   
1  Our hearts go out to all those affected by #Hu...      False   
2  Kissimmee neighborhood off of Michigan Ave. \n...      False   
3  I have this one tree in my backyard that scare...      False   
4  @AshleyRuizWx @Stephan89441722 @lilmizzheidi @...      False   

  display_text_range                                           entities  \
0            [0, 76]  {'hashtags': [{'text': 'H

In [138]:
dt_tweets.head(1)['entities']

0    {'hashtags': [{'text': 'HurricaneIan', 'indice...
Name: entities, dtype: object

## HINT
Take into account that for future queries, the final output must return (when
present) the following information for each of the selected documents: Tweet |
Username | Date | Hashtags | Likes | Retweets | Url (here the “Url” means the
tweet link).

## Marta: el url no he sapigut detectar quin era així q de moment ho he deixat buit... I els hastags no estic del tot segura el q volem...

In [174]:
tweets_query_format=pd.DataFrame(columns=['Tweet', 'Username', 'Date', 'Hashtags', 'Likes', 'Retweets', 'Url'])
for index, row in dt_tweets.iterrows():
    new_row = pd.DataFrame.from_dict({'Tweet': [row['full_text']], 'Username': [row['user']['name']], 'Date': [row['created_at']], 'Hashtags': [row['entities']['hashtags'][0]['text']], 'Likes': [row['favorite_count']], 'Retweets': [row['retweet_count']], 'Url': []}, orient='index').T
    tweets_query_format=pd.concat([new_row, tweets_query_format]).reset_index(drop=True)
print(tweets_query_format.head())

                                               Tweet              Username  \
0  This is awful.   https://t.co/BzhXaAPv7B￼\n\n#...      Ohemgeeitsalyssa   
1  Damage in my area in Punta Gorda...a thread. I...             CJ Haddad   
2  So it really wasn't #HurricaneIan that flooded...              @jganyfl   
3  Why isn’t @OsceolaCountyFl listed on the @fema...         BaconBitsNews   
4  The CFRD, @CarrboroPD , Carrboro Public Works,...  Carrboro Fire-Rescue   

                             Date      Hashtags Likes Retweets   Url  
0  Fri Sep 30 14:32:56 +0000 2022      DeSantis     0        0  None  
1  Fri Sep 30 14:32:56 +0000 2022  HurricaneIan     2        1  None  
2  Fri Sep 30 14:32:57 +0000 2022  HurricaneIan    16        8  None  
3  Fri Sep 30 14:33:01 +0000 2022     Kissimmee     0        0  None  
4  Fri Sep 30 14:33:06 +0000 2022  CarrboroSafe     2        0  None  


In [175]:
tweets_query_format

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,This is awful. https://t.co/BzhXaAPv7B￼\n\n#...,Ohemgeeitsalyssa,Fri Sep 30 14:32:56 +0000 2022,DeSantis,0,0,None
1,Damage in my area in Punta Gorda...a thread. I...,CJ Haddad,Fri Sep 30 14:32:56 +0000 2022,HurricaneIan,2,1,None
2,So it really wasn't #HurricaneIan that flooded...,@jganyfl,Fri Sep 30 14:32:57 +0000 2022,HurricaneIan,16,8,None
3,Why isn’t @OsceolaCountyFl listed on the @fema...,BaconBitsNews,Fri Sep 30 14:33:01 +0000 2022,Kissimmee,0,0,None
4,"The CFRD, @CarrboroPD , Carrboro Public Works,...",Carrboro Fire-Rescue,Fri Sep 30 14:33:06 +0000 2022,CarrboroSafe,2,0,None
...,...,...,...,...,...,...,...
3995,@AshleyRuizWx @Stephan89441722 @lilmizzheidi @...,Tess 💋,Fri Sep 30 18:38:53 +0000 2022,HurricaneIan,0,0,None
3996,I have this one tree in my backyard that scare...,alex ✨,Fri Sep 30 18:38:57 +0000 2022,scwx,0,0,None
3997,Kissimmee neighborhood off of Michigan Ave. \n...,Christopher Heath,Fri Sep 30 18:38:58 +0000 2022,HurricaneIan,0,0,None
3998,Our hearts go out to all those affected by #Hu...,Lytx,Fri Sep 30 18:39:01 +0000 2022,HurricaneIan,0,0,None


In [204]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line= line.lower() ## Transform in lowercase
    line=line.replace('@', '')
    line=line.replace('#', '')
    line=line.replace('.', '')
    line= line.split() ## Tokenize the text to get a list of terms
    line=[x for x in line if x not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line = filter(lambda x:x[0:5]!='https', line)
    line=[stemmer.stem(x) for x in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line

In [201]:
ss= "patata https://t.co/bzhxaapv7b"
print(" ".join(filter(lambda x:x[0:5]!='https', ss.split())))

patata


In [205]:
tweets_query_format_processed=pd.DataFrame(columns=['Tweet', 'Username', 'Date', 'Hashtags', 'Likes', 'Retweets', 'Url'])
tweets_query_format_processed = tweets_query_format.copy()
for index, row in tweets_query_format_processed.iterrows():
    tweets_query_format_processed.iloc[index]['Tweet']=build_terms(row['Tweet'])
print(tweets_query_format_processed.head())

                                               Tweet              Username  \
0  [aw, desanti, busi, scheme, traffick, asylum, ...      Ohemgeeitsalyssa   
1  [damag, area, punta, gordaa, thread, tropic, g...             CJ Haddad   
2  [realli, hurricaneian, flood, florida, magatea...              @jganyfl   
3  [isn’t, osceolacountyfl, list, fema, website?,...         BaconBitsNews   
4  [cfrd,, carrboropd, ,, carrboro, public, works...  Carrboro Fire-Rescue   

                             Date      Hashtags Likes Retweets   Url  
0  Fri Sep 30 14:32:56 +0000 2022      DeSantis     0        0  None  
1  Fri Sep 30 14:32:56 +0000 2022  HurricaneIan     2        1  None  
2  Fri Sep 30 14:32:57 +0000 2022  HurricaneIan    16        8  None  
3  Fri Sep 30 14:33:01 +0000 2022     Kissimmee     0        0  None  
4  Fri Sep 30 14:33:06 +0000 2022  CarrboroSafe     2        0  None  


In [18]:
print(build_terms("One day I saw a cow without a tie dressed with a uniform"))

['one', 'day', 'saw', 'cow', 'without', 'tie', 'dress', 'uniform']


In [81]:
def create_index(lines):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    title_index = {}  # dictionary to map page titles to page ids
    for line in lines:  # Remember, lines contain all documents: article-id | article-title | article-body
        #line_arr = line.split("\")
        #print(line.split("\t"))
        line_arr=line.split("\t")[1]
        page_id = int(line_arr)
        terms = build_terms(''.join(line_arr[1:])) # page_title + page_text
        title = line_arr[1]
        title_index[page_id]=title  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,3]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                
        ## START CODE
                current_page_index[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        ## END CODE                    
                    
    return index, title_index

In [83]:
start_time = time.time()
index, title_index = create_index(lines)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 3.61 seconds
